In [1]:
import numpy as np

In [2]:
testtype = np.dtype([("flag1","?"),("flag2","?")])

In [3]:
testtype

dtype([('flag1', '?'), ('flag2', '?')])

In [4]:
testtype.itemsize

2

In [5]:
np.dtype("i4, (2,3)f8")

dtype([('f0', '<i4'), ('f1', '<f8', (2, 3))])

In [6]:
dt1 = np.dtype([('timestamp', np.uint32),('imutimestamp', np.uint32), ('accelerometer', np.int16,3), ('gyro', np.int16,3), ('data', np.uint16,5*1),('reference',np.uint16)])

In [7]:
dt1.descr

[('timestamp', '<u4'),
 ('imutimestamp', '<u4'),
 ('accelerometer', '<i2', (3,)),
 ('gyro', '<i2', (3,)),
 ('data', '<u2', (5,)),
 ('reference', '<u2')]

In [8]:
dt2 = np.dtype(
            [('timestamp', 'f4'), ('level', 'u4'), ('channels', [('Cb', 'u1'), ('Cr', 'u1'), ('Y', 'u4')], (4)),
             ('accelerometer', 'f4', (3)), ('gyroscope', 'f4', (3))])

In [9]:
dt2

dtype([('timestamp', '<f4'), ('level', '<u4'), ('channels', [('Cb', 'u1'), ('Cr', 'u1'), ('Y', '<u4')], (4,)), ('accelerometer', '<f4', (3,)), ('gyroscope', '<f4', (3,))])

In [10]:
dt2.descr

[('timestamp', '<f4'),
 ('level', '<u4'),
 ('channels', [('Cb', '|u1'), ('Cr', '|u1'), ('Y', '<u4')], (4,)),
 ('accelerometer', '<f4', (3,)),
 ('gyroscope', '<f4', (3,))]

In [11]:
import json



In [12]:
json.dumps(dt2.descr)

'[["timestamp", "<f4"], ["level", "<u4"], ["channels", [["Cb", "|u1"], ["Cr", "|u1"], ["Y", "<u4"]], [4]], ["accelerometer", "<f4", [3]], ["gyroscope", "<f4", [3]]]'

In [13]:
dt2.descr

[('timestamp', '<f4'),
 ('level', '<u4'),
 ('channels', [('Cb', '|u1'), ('Cr', '|u1'), ('Y', '<u4')], (4,)),
 ('accelerometer', '<f4', (3,)),
 ('gyroscope', '<f4', (3,))]

In [14]:
np.dtype(">i4").descr

[('', '>i4')]

In [15]:
t = type(1)

In [16]:
def scalar_to_ctype(dt):
    # https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.dtypes.html
    
    ctypes = {
        "i4":"int32_t",
        "i2":"int16_t",
        "i1":"int8_t",
        "u4":"uint32_t",
        "u2":"uint16_t",
        "u1":"uint8_t",
        "f4":"float",
        "f8":"double",
    }
    
    endianes = dt[0]
    t = dt[1:]
    
    if endianes not in "|=><":
        t = dt
        endianes = "="
    
    typecode = t[0]
    size = 1
    if len(t) > 1:
        size = int(t[1])
    
    ctype = ctypes.get(t);
    print(dt,endianes,t,typecode,size,ctype)
    return ctype
    
scalar_to_ctype(np.dtype(">i4").descr[0][1])
scalar_to_ctype(np.dtype(">i").descr[0][1])
scalar_to_ctype(np.dtype("i").descr[0][1])
scalar_to_ctype(np.dtype("<i2").descr[0][1])
scalar_to_ctype(np.dtype(">u4").descr[0][1])
scalar_to_ctype(np.dtype(">u2").descr[0][1])
scalar_to_ctype(np.dtype("u1").descr[0][1])
scalar_to_ctype(np.dtype("<u2").descr[0][1])
scalar_to_ctype(np.dtype("b").descr[0][1])
scalar_to_ctype(np.dtype("B").descr[0][1])
scalar_to_ctype(np.dtype("=i").descr[0][1])
scalar_to_ctype(np.dtype("?").descr[0][1])


dt1

('>i4', '>', 'i4', 'i', 4, 'int32_t')
('>i4', '>', 'i4', 'i', 4, 'int32_t')
('<i4', '<', 'i4', 'i', 4, 'int32_t')
('<i2', '<', 'i2', 'i', 2, 'int16_t')
('>u4', '>', 'u4', 'u', 4, 'uint32_t')
('>u2', '>', 'u2', 'u', 2, 'uint16_t')
('|u1', '|', 'u1', 'u', 1, 'uint8_t')
('<u2', '<', 'u2', 'u', 2, 'uint16_t')
('|i1', '|', 'i1', 'i', 1, 'int8_t')
('|u1', '|', 'u1', 'u', 1, 'uint8_t')
('<i4', '<', 'i4', 'i', 4, 'int32_t')
('|b1', '|', 'b1', 'b', 1, None)


dtype([('timestamp', '<u4'), ('imutimestamp', '<u4'), ('accelerometer', '<i2', (3,)), ('gyro', '<i2', (3,)), ('data', '<u2', (5,)), ('reference', '<u2')])

In [17]:

    
def cstruct_from_dtype(dt):
    cstruct = []
    if hasattr(dt,"descr"):
        descr = dt.descr
    else:
        descr = dt
        
#     print(descr)
    for item in descr:
#         print(item)
        name = item[0]
        t = item[1]
        shape = None
        
        if len(item) > 2:
            shape = item[2]
        
        if isinstance(t,list):
            ctype = cstruct_from_dtype(t)
        else:
            ctype = scalar_to_ctype(t)
            
#         print(name,t,ctype)
        cstruct.append([name,ctype,shape])
    return cstruct


struct = cstruct_from_dtype(dt1)
struct

('<u4', '<', 'u4', 'u', 4, 'uint32_t')
('<u4', '<', 'u4', 'u', 4, 'uint32_t')
('<i2', '<', 'i2', 'i', 2, 'int16_t')
('<i2', '<', 'i2', 'i', 2, 'int16_t')
('<u2', '<', 'u2', 'u', 2, 'uint16_t')
('<u2', '<', 'u2', 'u', 2, 'uint16_t')


[['timestamp', 'uint32_t', None],
 ['imutimestamp', 'uint32_t', None],
 ['accelerometer', 'int16_t', (3,)],
 ['gyro', 'int16_t', (3,)],
 ['data', 'uint16_t', (5,)],
 ['reference', 'uint16_t', None]]

In [2]:
struct2 = cstruct_from_dtype(dt2)
struct2

NameError: name 'cstruct_from_dtype' is not defined

In [19]:
def format_cstruct(cstruct, name=None, indent=4):
    out = "struct {\n"
    def format_declaration(name,type_,shape):
        #print(type_,name)
        if isinstance(type_,list):
            type_= format_cstruct(type_)
            print(type)
        
        type_ = "\n".join(map(lambda line: " "*indent + line ,  type_.split("\n")))
        
        if shape is not None:
            name = name + "".join(map(lambda d: "[" + str(d) + "]",list(shape)))
        
        return type_ + " " + name +";";
        
    out = out + "\n".join(map( lambda d: format_declaration(*d) , cstruct ))
    
    out = out +  " "*indent + "\n}" 
    if name is not None:
        out = out + " " + name 
    out += ";"
    return out
    
 
c = format_cstruct(struct,name="test")
print(c)

struct {
    uint32_t timestamp;
    uint32_t imutimestamp;
    int16_t accelerometer[3];
    int16_t gyro[3];
    uint16_t data[5];
    uint16_t reference;    
} test;


In [20]:
c = format_cstruct(struct2)
print(c)

<type 'type'>
struct {
    float timestamp;
    uint32_t level;
    struct {
        uint8_t Cb;
        uint8_t Cr;
        uint32_t Y;    
    }; channels[4];
    float accelerometer[3];
    float gyroscope[3];    
};


In [21]:
a = np.array([1,2,3])

In [22]:
r = np.rec.array(a,dtype=np.dtype([("test","f4")]))
r

rec.array([(  1.40129846e-45,), (  0.00000000e+00,), (  2.80259693e-45,),
 (  0.00000000e+00,), (  4.20389539e-45,), (  0.00000000e+00,)], 
          dtype=[('test', '<f4')])

In [23]:
r["test"]

array([  1.40129846e-45,   0.00000000e+00,   2.80259693e-45,
         0.00000000e+00,   4.20389539e-45,   0.00000000e+00], dtype=float32)

In [24]:
np.save("test.npy",r,fortran_order=1)

TypeError: save() got an unexpected keyword argument 'fortran_order'

In [25]:
with open("test.npy","rb") as b:
    npy = b.read()
    

In [26]:
npy

"\x93NUMPY\x01\x00V\x00{'descr': [('test', '<f4')], 'fortran_order': False, 'shape': (6,), }                \n\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00"

In [27]:
c = """
#define AZHRSignalGrid 2


typedef struct{
    float timestamp;
    uint32_t level;
    AZHRColorChannel_t channels[AZHRSignalGrid*AZHRSignalGrid];
    float accelerometer[3];
    float gyroscope[3];
} AZHRSignal_t;

"""

In [34]:
print(c)


#define AZHRSignalGrid 2


typedef struct{
    float timestamp;
    uint32_t level;
    AZHRColorChannel_t channels[AZHRSignalGrid*AZHRSignalGrid];
    float accelerometer[3];
    float gyroscope[3];
} AZHRSignal_t;




"{'1': (1,)}"